In [1]:
# Import statements
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import acf

In [2]:
cases20 = pd.read_csv('Data/newCasesPCROnlyBySpecimenDate_ltla_2020.csv')
cases20 = cases20[['date','area_code','area_name','value']]

In [6]:
cases20[cases20['area_name']=='York']

,date,area_code,area_name,value
161,2020-06-01,E06000014,York,3.0
495,2020-06-02,E06000014,York,7.0
802,2020-06-03,E06000014,York,2.0
1011,2020-06-04,E06000014,York,1.0
1403,2020-06-05,E06000014,York,3.0
...,...,...,...,...
56212,2020-12-27,E06000014,York,128.0
56551,2020-12-28,E06000014,York,133.0
56756,2020-12-29,E06000014,York,220.0
57102,2020-12-30,E06000014,York,186.0


In [3]:
ltlacovid = np.unique(cases20['area_name'])

In [4]:
lad19 = pd.read_csv('Data/England_LTLA_NHSER.csv')
# ltlalad = np.unique(lad19['LAD19NM'])

In [8]:
nhsltlas = np.unique(lad19['LTLA_name'])
nhsltlas

array(['Adur', 'Allerdale', 'Amber Valley', 'Arun', 'Ashfield', 'Ashford',
       'Babergh', 'Barking and Dagenham', 'Barnet', 'Barnsley',
       'Barrow-in-Furness', 'Basildon', 'Basingstoke and Deane',
       'Bassetlaw', 'Bath and North East Somerset', 'Bedford', 'Bexley',
       'Birmingham', 'Blaby', 'Blackburn with Darwen', 'Blackpool',
       'Bolsover', 'Bolton', 'Boston',
       'Bournemouth, Christchurch and Poole', 'Bracknell Forest',
       'Bradford', 'Braintree', 'Breckland', 'Brent', 'Brentwood',
       'Brighton and Hove', 'Bristol, City of', 'Broadland', 'Bromley',
       'Bromsgrove', 'Broxbourne', 'Broxtowe', 'Buckinghamshire',
       'Burnley', 'Bury', 'Calderdale', 'Cambridge', 'Camden',
       'Cannock Chase', 'Canterbury', 'Carlisle', 'Castle Point',
       'Central Bedfordshire', 'Charnwood', 'Chelmsford', 'Cheltenham',
       'Cherwell', 'Cheshire East', 'Cheshire West and Chester',
       'Chesterfield', 'Chichester', 'Chorley', 'City of London',
       'Colch

In [9]:
set(nhsltlas) - set(ltlacovid)

{'Buckinghamshire', 'City of London', 'Cornwall', 'Hackney', 'Isles of Scilly'}

In [17]:
lad19[lad19['LTLA_name']=='City of London']

,Unnamed: 0,LTLA_code,LTLA_name,NHSER_code,NHSER_name,idNHSER,combinedPop_allAges
281,282,E09000001,City of London,E40000003,London,1,8776.4


In [13]:
lad19[lad19['LTLA_name']=='Isles of Scilly']

,Unnamed: 0,LTLA_code,LTLA_name,NHSER_code,NHSER_name,idNHSER,combinedPop_allAges
49,50,E06000053,Isles of Scilly,E40000006,South West,3,1998.9


In [16]:
lad19[lad19['LTLA_name']=='Buckinghamshire']

,Unnamed: 0,LTLA_code,LTLA_name,NHSER_code,NHSER_name,idNHSER,combinedPop_allAges
56,57,E06000060,Buckinghamshire,E40000005,South East,2,553100.0


In [10]:
set(ltlacovid) - set(nhsltlas)

{'Aylesbury Vale',
 'Chiltern',
 'Cornwall and Isles of Scilly',
 'Hackney and City of London',
 'South Bucks',
 'Wycombe'}

In [6]:
ltlacovid

array(['Adur', 'Allerdale', 'Amber Valley', 'Arun', 'Ashfield', 'Ashford',
       'Aylesbury Vale', 'Babergh', 'Barking and Dagenham', 'Barnet',
       'Barnsley', 'Barrow-in-Furness', 'Basildon',
       'Basingstoke and Deane', 'Bassetlaw',
       'Bath and North East Somerset', 'Bedford', 'Bexley', 'Birmingham',
       'Blaby', 'Blackburn with Darwen', 'Blackpool', 'Bolsover',
       'Bolton', 'Boston', 'Bournemouth, Christchurch and Poole',
       'Bracknell Forest', 'Bradford', 'Braintree', 'Breckland', 'Brent',
       'Brentwood', 'Brighton and Hove', 'Bristol, City of', 'Broadland',
       'Bromley', 'Bromsgrove', 'Broxbourne', 'Broxtowe', 'Burnley',
       'Bury', 'Calderdale', 'Cambridge', 'Camden', 'Cannock Chase',
       'Canterbury', 'Carlisle', 'Castle Point', 'Central Bedfordshire',
       'Charnwood', 'Chelmsford', 'Cheltenham', 'Cherwell',
       'Cheshire East', 'Cheshire West and Chester', 'Chesterfield',
       'Chichester', 'Chiltern', 'Chorley', 'Colchester', 'Copel

In [34]:
# Convert to wide format for ease of future calculations
wide_data_total = weekly_counts_total.pivot(index='date', columns='area_name', values='detrended')
wide_data_incidence = weekly_counts_total.pivot(index='date', columns='area_name', values='incidence')